# AI Agent for Contructing the Prompt to another AI Agent

Using Langchain Library with LLaMA

## Install LLaMA with LangChain

In [6]:
pip install langchain transformers torch git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/tn/8_1x_bp94r1c1stsyybjfgt80000gn/T/pip-req-build-7r78jgk8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/tn/8_1x_bp94r1c1stsyybjfgt80000gn/T/pip-req-build-7r78jgk8
  Resolved https://github.com/huggingface/transformers.git to commit 78b2929c0554b79e0489b451ce4ece14d265ead2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9786808 sha256=94459b6bff9033e9b32b6c1787cf1b69dfc0f0b1415272094e8cbe2b9c8fc626
  Stored in directory: /private/var/folders/tn/8_1x_bp94r1c1stsyybjfgt80000gn/T/pip-ephem-wheel-cache-tkjer0ei/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: transformers
    Found exis

## Loading LLaMA Model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer from Hugging Face (if available)
model_name = "huggingface/llama-7b"  # Ensure this is a correct path to a LLaMA model in Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


OSError: huggingface/llama-7b is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [7]:
from langchain_openai import ChatOpenAI
from openai import OpenAI

client = OpenAI(
  organization='org-4qgscp2CoIfa5lzkHj52zIN6',
  project='$PROJECT_ID',
)

metrics = {
    "News and Announcements": {
        "importance_weight": 1.5,  # High importance
        "rubric": """
        - **Grade -2:** **Highly Negative Impact**
        - **Criteria:** Significant negative news that can severely harm the company's growth prospects.
        - **Examples:** Major scandals, lawsuits, regulatory penalties, bankruptcy filings, data breaches affecting a large number of customers.

        - **Grade -1:** **Moderately Negative Impact**
        - **Criteria:** Negative news that could impede growth but is not necessarily catastrophic.
        - **Examples:** Product recalls, executive resignations under controversy, loss of key clients, minor legal disputes.

        - **Grade 0:** **Neutral Impact**
        - **Criteria:** News that is routine or has no significant effect on growth.
        - **Examples:** Regular updates, minor staffing changes, routine financial reports, small-scale operational adjustments.

        - **Grade 1:** **Moderately Positive Impact**
        - **Criteria:** Positive news that may contribute to growth.
        - **Examples:** New product features, minor partnerships, awards, positive quarterly earnings exceeding expectations.

        - **Grade 2:** **Highly Positive Impact**
        - **Criteria:** Significant positive news likely to substantially boost growth.
        - **Examples:** Major acquisitions, entering new markets, groundbreaking product launches, significant investment funding.
        """,
        "example": 
            """text: Company X acquired a major competitor.
            grade: 2
            justification: A major acquisition likely to significantly boost growth.""",
        "data": [  # Data set for this metric
            "Company X launched a new product line.",
            "Company X had to recall a product that was found to be defective."
            # Add more data entries
        ],
    },
    "Events": {
        "importance_weight": 1,  # Medium importance
        "rubric": """
        - **Grade -2:** **Highly Negative Impact**
        - **Criteria:** Events that result in significant negative publicity or reputational harm, potentially damaging the company’s growth.
        - **Examples:** Event cancellations due to poor management, incidents causing injuries or legal issues at company-hosted events.

        - **Grade -1:** **Moderately Negative Impact**
        - **Criteria:** Events that may impede growth due to minor negative publicity or operational issues.
        - **Examples:** Low attendance at key events, technical failures during important presentations, negative feedback from event participants.

        - **Grade 0:** **Neutral Impact**
        - **Criteria:** Events that have no significant effect on growth, such as routine participation in industry conferences with no notable outcomes.
        - **Examples:** Participation in routine industry conferences or small-scale webinars without any major positive or negative outcomes.

        - **Grade 1:** **Moderately Positive Impact**
        - **Criteria:** Events that contribute to growth through increased visibility, networking, or improved brand perception.
        - **Examples:** Successful product demonstrations at trade shows, positive engagement at industry conferences, hosting well-received webinars or workshops.

        - **Grade 2:** **Highly Positive Impact**
        - **Criteria:** Events that are likely to substantially boost growth through significant positive exposure, high-profile partnerships, or substantial media coverage.
        - **Examples:** Hosting major international conferences, delivering keynote speeches that lead to new partnerships, events resulting in widespread media attention or industry acclaim.
        """,
        "example":
            """text: Company X won an industry award.
            grade: 2
            justification: Winning an award enhances reputation and growth potential.""",
        "data": [  # Data set for this metric
            "Company X hosted a successful conference.",
            "Company X presented a new technology.",
            "Company X had low attendance at a conference."
            # Add more data entries
        ],
    },
    # Add more metrics as needed
}

UNIVERSAL_STATEMENT = """
You are an analyst assessing companies for their growth potential based on various metrics.
Your task is to evaluate the provided information and assign a grade from -2 to 2,
where negative values indicate negative impact on growth, and positive values indicate positive impact.
Provide a brief justification for the grade based on the criteria.

You are to answer in this format:

Grade: **insert grade**\n\n
Justification: **insert justification**
"""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)  # Use a low temperature for consistency

def evaluate_metric(metric_name, metric_config):
    # Extract configurations
    importance_weight = metric_config["importance_weight"]
    rubric = metric_config["rubric"]
    example = metric_config["example"]
    data_entries = metric_config["data"]
    # Construct the prompt template
    prompt_template = (
        UNIVERSAL_STATEMENT
        + f"\n**Metric:** {metric_name}\n"
        + rubric
        + "\n**Example:**"
        + example
        + "\n---\n**Now, evaluate the following text:**\n\nText:\n\"{input_text}\""
    )
    
    results = []
    for input_text in data_entries:
        # Format the prompt with the input text
        prompt = prompt_template.format(input_text=input_text)
        # Get the LLM response
        print(prompt_template)
        response = llm.invoke(prompt)
        # Parse the grade and justification
        lines = response.content.strip().split("\n\n")
        grade = lines[0].replace("Grade:", "").strip()
        justification = lines[1].replace("Justification:", "").strip()
        # Append the result
        results.append({
            "metric": metric_name,
            "text": input_text,
            "grade": float(grade),
            "justification": justification,
            "importance_weight": importance_weight
        })
    return results

all_results = []

for metric_name, metric_config in metrics.items():
    print(f"Evaluating metric: {metric_name}")
    metric_results = evaluate_metric(metric_name, metric_config)
    all_results.extend(metric_results)

overall_score = 0
max_score = 0

for result in all_results:
    grade = result['grade']
    weight = result['importance_weight']
    print(f"Calculating Score for {result['metric']}, with grade {grade} and weight {weight}")
    overall_score += grade * weight
    max_score += 2 * weight

# Calculate the normalized overall growth indicator
overall_growth_indicator = overall_score / max_score

print(f"Overall Score: {overall_score}, Max Score: {max_score}")
print(f"Overall Growth Indicator: {overall_growth_indicator:.2f}")

print(all_results)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable